<a href="https://colab.research.google.com/github/kaipak/from-eyes-to-ears/blob/master/mask_training_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!pip install numpy scikit-image Shapely awscli

In [0]:
import pandas as pd
import os
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from skimage import measure
from shapely.geometry import Polygon, MultiPolygon
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import pickle

In [37]:
!rm -rf train validation test *.zip
!mkdir tmp
!mkdir train
!mkdir train/images
!mkdir validation
!mkdir validation/images
!mkdir test
!mkdir test/images
!mkdir masks/

mkdir: cannot create directory ‘tmp’: File exists
mkdir: cannot create directory ‘masks/’: File exists


In [0]:
phase = 'train'
filter_categories = ["Saxophone", "Tank", "Goose", "Guitar"]

## Run the cell that match the current phase

In [64]:
!rm *.csv
!wget https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv
!wget https://storage.googleapis.com/openimages/v5/train-annotations-human-imagelabels-boxable.csv
!rm -rf masks/
!mkdir masks/
%cd masks/
!wget https://storage.googleapis.com/openimages/v5/train-masks/train-masks-0.zip
!unzip train-masks-0.zip > /dev/null
!wget https://storage.googleapis.com/openimages/v5/train-masks/train-masks-1.zip
!unzip train-masks-1.zip > /dev/null
%cd ..

--2020-05-06 16:33:18--  https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.217.128, 2607:f8b0:400c:c13::80
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.217.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12011 (12K) [text/csv]
Saving to: ‘class-descriptions-boxable.csv’

class-descriptions- 100%[===================>]  11.73K  --.-KB/s    in 0s      

2020-05-06 16:33:18 (99.6 MB/s) - ‘class-descriptions-boxable.csv’ saved [12011/12011]

--2020-05-06 16:33:19--  https://storage.googleapis.com/openimages/v5/train-annotations-human-imagelabels-boxable.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.12.128, 2607:f8b0:400c:c15::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.12.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 376764810 (359M) [text/csv]
Saving to: ‘

In [47]:
!rm *.csv
!wget https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv
!wget https://storage.googleapis.com/openimages/v5/validation-annotations-human-imagelabels-boxable.csv
!rm -rf masks/
!mkdir masks/
%cd masks/
!wget https://storage.googleapis.com/openimages/v5/validation-masks/validation-masks-0.zip
!unzip validation-masks-0.zip > /dev/null
!wget https://storage.googleapis.com/openimages/v5/validation-masks/validation-masks-1.zip
!unzip validation-masks-1.zip > /dev/null
%cd ..

--2020-05-06 16:15:52--  https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.203.128, 2607:f8b0:400c:c15::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12011 (12K) [text/csv]
Saving to: ‘class-descriptions-boxable.csv’

class-descriptions- 100%[===================>]  11.73K  --.-KB/s    in 0s      

2020-05-06 16:15:52 (102 MB/s) - ‘class-descriptions-boxable.csv’ saved [12011/12011]

--2020-05-06 16:15:53--  https://storage.googleapis.com/openimages/v5/validation-annotations-human-imagelabels-boxable.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.204.128, 2607:f8b0:400c:c15::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10649275 (10M) [text/csv]
Saving t

In [55]:
!rm *.csv
!wget https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv
!wget https://storage.googleapis.com/openimages/v5/test-annotations-human-imagelabels-boxable.csv
!rm -rf masks/
!mkdir masks/
%cd masks/
!wget https://storage.googleapis.com/openimages/v5/test-masks/test-masks-0.zip
!unzip test-masks-0.zip > /dev/null
!wget https://storage.googleapis.com/openimages/v5/test-masks/test-masks-1.zip
!unzip test-masks-1.zip > /dev/null
%cd ..

--2020-05-06 16:16:37--  https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.217.128, 2607:f8b0:400c:c0a::80
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.217.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12011 (12K) [text/csv]
Saving to: ‘class-descriptions-boxable.csv’

class-descriptions- 100%[===================>]  11.73K  --.-KB/s    in 0s      

2020-05-06 16:16:37 (77.1 MB/s) - ‘class-descriptions-boxable.csv’ saved [12011/12011]

--2020-05-06 16:16:38--  https://storage.googleapis.com/openimages/v5/test-annotations-human-imagelabels-boxable.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.217.128, 2607:f8b0:400c:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.217.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32055857 (31M) [text/csv]
Saving to: ‘t

In [0]:
classes = pd.read_csv('class-descriptions-boxable.csv', header=None, names=["LabelName", "class"])
image_classes = pd.read_csv(f'{phase}-annotations-human-imagelabels-boxable.csv')

In [0]:
# Find where we have data
training_imageids = classes.join(image_classes.set_index('LabelName'), on='LabelName').filter(["ImageID", "LabelName","class"])
training_imageids = training_imageids[training_imageids["ImageID"].str.startswith('0') | training_imageids["ImageID"].str.startswith('1')]
training_imageids['cutLabelName'] = training_imageids["LabelName"].str.replace('/', '')
masks = pd.DataFrame([f for f in [file.split('_') + [file] for file in os.listdir('masks')] if len(f) == 4], columns=['ImageID', 'cutLabelName', 'extension', 'mask_filename'])
training_imageids = training_imageids.merge(masks,  how='inner', left_on=['ImageID', 'cutLabelName'], right_on = ['ImageID', 'cutLabelName']).filter(["ImageID", "LabelName", "class", 'mask_filename'])

## Get a summary to pick classes for training

In [67]:
i = pd.get_option('display.max_rows', None)
pd.set_option('display.max_rows', None)
print(training_imageids.groupby("class").count()[['ImageID']])
pd.set_option('display.max_rows', i)

                         ImageID
class                           
Adhesive tape                 62
Aircraft                     217
Airplane                    2704
Alarm clock                   30
Alpaca                       143
Ambulance                    120
Apple                        521
Armadillo                      1
Artichoke                     40
Axe                           31
Backpack                     421
Balance beam                  58
Ball                         497
Balloon                     1114
Banana                       267
Band-aid                       2
Barge                        119
Barrel                       345
Baseball bat                 260
Baseball glove               331
Bat (Animal)                 121
Beaker                        36
Bear                          62
Beer                        1060
Bell pepper                  154
Belt                          68
Bicycle wheel               6542
Billiard table               227
Bird      

In [68]:
filtered = training_imageids[training_imageids["class"].isin(filter_categories)].filter(["ImageID", "mask_filename", "class"])
filtered["info"] = list(zip(filtered['mask_filename'], filtered['class']))
masks = filtered.groupby("ImageID")["info"].apply(list).values.tolist()
print(len(masks))

3370


In [69]:
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
categories = []
metadata = []
max_size = 500
cat_counts = {}
n = 1
for m in masks:
    annotations = []
    catset = set()
    for f, c in m:
        catset.add(c)
    
    cont = False
    for c in catset:
        if c in cat_counts:
            if cat_counts[c] > max_size:
                cont = True
                break
            else:
                cat_counts[c] += 1
        else:
            cat_counts[c] = 1
    
    if cont:
        #print(f"we have collected enough for categores {catset}")
        continue

    for file, category in m:
        if category not in categories:
            categories.append(category)

        category_id = categories.index(category)
        fullname = f"masks/{file}"
        image_id = file.split('_')[0]
        mask = (cv2.imread(fullname)[...,0]/255.0).astype(np.uint8)
        contours = measure.find_contours(mask, 0.5, positive_orientation='low')
        segmentations = []
        polygons = []
        for contour in contours:
            # Flip from (row, col) representation to (x, y)
            # and subtract the padding pixel
            for i in range(len(contour)):
                row, col = contour[i]
                contour[i] = (col - 1, row - 1)
            try:
                # Make a polygon and simplify it
                poly = Polygon(contour)
                poly = poly.simplify(1.0, preserve_topology=False)
                if poly.type == 'MultiPolygon':
                    for p in poly:
                        polygons.append(p)
                        segmentation = np.array(p.exterior.coords).ravel().tolist()
                        if len(segmentation) >= 6:
                            segmentations.append(segmentation)
                        else:
                            continue
                else:
                    polygons.append(poly)
                    segmentation = np.array(poly.exterior.coords).ravel().tolist()
                    if len(segmentation) >= 6:
                        segmentations.append(segmentation)
                    else:
                        continue
            except:
                print("Error processing image mask, skipping")
                continue
        
        if len(segmentations) == 0:
            print("Can't build polygons, skipping")
            continue

        # Combine the polygons to calculate the bounding box and area
        multi_poly = MultiPolygon(polygons)
        x, y, max_x, max_y = multi_poly.bounds
        width = max_x - x
        height = max_y - y
        bbox = (x, y, width, height)
        area = multi_poly.area

        annotation = {
            'bbox': multi_poly.bounds,
            'bbox_mode': 0,
            'category_id': category_id,
            'iscrowd': 0,
            'segmentation': segmentations
        }
        annotations.append(annotation)

    if len(annotations) > 0:
        h, w = mask.shape
        md = {
            'annotations': annotations,
            'file_name': f'{phase}/images/{image_id}.jpg',
            'height': h,
            'width': w,
            'image_id': image_id
        }
        metadata.append(md)

        image_file = f'{image_id}.jpg'
        print(f"{n} - Fetching image: {image_file} - Categories: {catset} - Current Counters: {cat_counts}")
        s3.download_file('open-images-dataset', f'{phase}/{image_file}', f'tmp/{image_file}')
        img = cv2.imread(f'tmp/{image_file}')
        os.unlink(f'tmp/{image_file}')
        img = cv2.resize(img, (w, h))
        cv2.imwrite(f'{phase}/images/{image_file}', img)
        n += 1


print(f"Data ready for phase: {phase}")
print("Encoding Metadata")
pickle.dump({"categories": categories, "metadata": metadata}, open(f"{phase}/metadata.pkl", "wb"))
print("Done")

1 - Fetching image: 0000201cd362f303.jpg - Categories: {'Guitar'} - Current Counters: {'Guitar': 1}
2 - Fetching image: 0000aa810854dc2e.jpg - Categories: {'Guitar'} - Current Counters: {'Guitar': 2}
3 - Fetching image: 0000b3e5921ab7ff.jpg - Categories: {'Guitar'} - Current Counters: {'Guitar': 3}
4 - Fetching image: 0000c33c6f4b8518.jpg - Categories: {'Guitar'} - Current Counters: {'Guitar': 4}
5 - Fetching image: 0001ff6b8fc43d43.jpg - Categories: {'Goose'} - Current Counters: {'Guitar': 4, 'Goose': 1}
6 - Fetching image: 00027ae34cb09507.jpg - Categories: {'Guitar'} - Current Counters: {'Guitar': 5, 'Goose': 1}
7 - Fetching image: 0002a3c01c926a49.jpg - Categories: {'Guitar'} - Current Counters: {'Guitar': 6, 'Goose': 1}
8 - Fetching image: 00034cd6adabef6a.jpg - Categories: {'Guitar'} - Current Counters: {'Guitar': 7, 'Goose': 1}
9 - Fetching image: 00036d5834d932bf.jpg - Categories: {'Guitar'} - Current Counters: {'Guitar': 8, 'Goose': 1}
10 - Fetching image: 0004c03cb6520680.jpg

In [70]:
!rm train.zip
!zip -r train.zip train 
!ls -lh

  adding: train/ (stored 0%)
  adding: train/metadata.pkl (deflated 77%)
  adding: train/images/ (stored 0%)
  adding: train/images/0e64d4a61b802d3c.jpg (deflated 0%)
  adding: train/images/1af60feda5a2204f.jpg (deflated 0%)
  adding: train/images/12f357388db95c18.jpg (deflated 0%)
  adding: train/images/0ff8e99d63f1e179.jpg (deflated 0%)
  adding: train/images/002c1766a931beef.jpg (deflated 0%)
  adding: train/images/005ba732f68b2949.jpg (deflated 0%)
  adding: train/images/19ae1dcf39ad3c69.jpg (deflated 0%)
  adding: train/images/06db68e4857c8be8.jpg (deflated 0%)
  adding: train/images/00396b5af34e4c0c.jpg (deflated 1%)
  adding: train/images/05cba30e171f7fec.jpg (deflated 0%)
  adding: train/images/050ddb50a117d98b.jpg (deflated 1%)
  adding: train/images/143605dbb7f07d3b.jpg (deflated 0%)
  adding: train/images/01807eda5be82ecd.jpg (deflated 2%)
  adding: train/images/00d43f1f970e9847.jpg (deflated 0%)
  adding: train/images/15f6bb4282e848fe.jpg (deflated 0%)
  adding: train/image

In [71]:
from google.colab import drive
drive.mount('/content/drive')
!cp *.zip "drive/My Drive"
drive.flush_and_unmount()

Mounted at /content/drive
